# Задача искусственного муравья

In [ ]:
import random
random.seed(42)

class Machine(object):

    def __init__(self, states, init_state, input_symbols, output_symbols, transition=None, action=None):

        self.states = states
        self.input_symbols = input_symbols
        self.output_symbols = output_symbols
        self.transition = transition or self.create_random_transition()
        self.action = action or self.create_random_action()
        self.init_state = init_state
        self.current_state = self.init_state

    @staticmethod
    def create_random(states_count, init_state, input_symbols, output_symbols):

        states = list(range(states_count))
        transition = {state: {symbol: random.choice(states) for symbol in input_symbols} for state in states}
        action = {state: {symbol: random.choice(output_symbols) for symbol in input_symbols} for state in states}

        return Machine(states, init_state, input_symbols, output_symbols, transition, action)

    def mutate(self, mutation_rate):

        for state in self.states:
            for symbol in self.input_symbols:
                if random.random() < mutation_rate:
                    self.transition[state][symbol] = random.choice(self.states)
                    self.action[state][symbol] = random.choice(self.output_symbols)

    @staticmethod
    def crossover(parent1, parent2):

        transition = {state: {symbol: random.choice([parent1.transition[state][symbol], parent2.transition[state][symbol]]) for symbol in parent1.input_symbols} for state in parent1.states}
        action = {state: {symbol: random.choice([parent1.action[state][symbol], parent2.action[state][symbol]]) for symbol in parent1.input_symbols} for state in parent1.states}
        init_state = 0
        child = Machine(parent1.states, init_state, parent1.input_symbols, parent1.output_symbols, transition, action)

        return child

    @staticmethod
    def evaluate(machine):

        max_moves = 900
        ant = AntSimulator(max_moves = max_moves)
        with open("map2.txt", 'r') as trial_file:
            ant.parse_matrix(trial_file)
        eaten_food = Machine.run_with_fsm(ant, machine)

        if eaten_food == ant.total_food:
          fitness = ant.moves
        else:
          fitness = max_moves + (ant.total_food - eaten_food)

        return fitness

    @staticmethod
    def run_with_fsm(antsimulator, machine):

      while antsimulator.moves < antsimulator.max_moves and antsimulator.eaten < antsimulator.total_food:

          input_ = InputSymbol.FOUND if antsimulator.sense_food() else InputSymbol.NOTFOUND
          output = machine.input_(input_)
          if output == OutputSymbol.FORWARD:
              antsimulator.move_forward()
          elif output == OutputSymbol.LEFT:
              antsimulator.turn_left()
          elif output == OutputSymbol.RIGHT:
              antsimulator.turn_right()

      eaten_food = antsimulator.eaten
      return eaten_food

    def input_(self, input_symbol):

        current_state = self.current_state
        end_state = self.transition[current_state][input_symbol]
        output_symbol = self.action[current_state][input_symbol]
        self.current_state = end_state

        return output_symbol

    def print_transition_table(self):

        print("===================================================")

        input_symbols_str = [str(i) for i in self.input_symbols]
        print("\t", "|\t".join(input_symbols_str))

        print("---------------------------------------------------")

        for state in self.states:

            end_states = self.transition[state].values()
            output_symbols = self.action[state].values()
            end_states_str = [str(state) for state in end_states]
            output_symbols_str = [str(i) for i in output_symbols]
            endstate_with_output = [end_states_str[i] + "/" + output_symbols_str[i] for i in range(len(end_states_str))]
            print(str(state), "|", "\t", "|\t".join(endstate_with_output))

        print("---------------------------------------------------")
        print("===================================================")

In [ ]:
import copy

class AntSimulator(object):

    direction = ["north", "east", "south", "west"]
    dir_row = [1, 0, -1, 0]
    dir_col = [0, 1, 0, -1]

    def __init__(self, max_moves):

        self.max_moves = max_moves
        self.moves = 0
        self.eaten = 0
        self.total_food = 0

    @property
    def position(self):

        return (self.row, self.col, self.direction[self.dir])

    def turn_left(self):

        if self.moves < self.max_moves:

            self.moves += 1
            self.dir = (self.dir - 1) % 4

    def turn_right(self):

        if self.moves < self.max_moves:

            self.moves += 1
            self.dir = (self.dir + 1) % 4

    def move_forward(self):

        if self.moves < self.max_moves:

            self.moves += 1
            self.row = (self.row + self.dir_row[self.dir]) % self.matrix_row
            self.col = (self.col + self.dir_col[self.dir]) % self.matrix_col
            if self.matrix_exc[self.row][self.col] == "food":
                self.eaten += 1
            self.matrix_exc[self.row][self.col] = "passed"

    def sense_food(self):

        ahead_row = (self.row + self.dir_row[self.dir]) % self.matrix_row
        ahead_col = (self.col + self.dir_col[self.dir]) % self.matrix_col

        return self.matrix_exc[ahead_row][ahead_col] == "food"

    def parse_matrix(self, matrix): # преобразует входную матрицу во внутреннее представление симулятора,
                                    # где '1' представляет еду, '0' - пустое пространство, 'S' - начальную позицию.
        self.matrix = list()
        for i, line in enumerate(matrix):
            self.matrix.append(list())
            for j, col in enumerate(line):
                if col == "1":
                    self.matrix[-1].append("food")
                    self.total_food += 1
                elif col == "0":
                    self.matrix[-1].append("empty")
                elif col == "S":
                    self.matrix[-1].append("empty")
                    self.row_start = self.row = i
                    self.col_start = self.col = j
                    self.dir = 1

        self.matrix_row = len(self.matrix)
        self.matrix_col = len(self.matrix[0])
        self.matrix_exc = copy.deepcopy(self.matrix)

    def print_route(self): # вывод карты симуляции, где '1' - еда, '0' - пустое пространство, '*' - прошедший путь.

        for row in self.matrix_exc:
            line = list()
            for col in row:
                if col == "food":
                    line.append("1")
                elif col == "empty":
                    line.append("0")
                elif col == "passed":
                    line.append("*")
            print(''.join(line))

    def write_route(self, filename):

        with open(filename, 'w') as outputfile:
            for row in self.matrix_exc:
                line = list()
                for col in row:
                    if col == "food":
                        line.append("1")
                    elif col == "empty":
                        line.append("0")
                    elif col == "passed":
                        line.append("*")
                outputfile.write(''.join(line) + "\n")

In [ ]:
from enum import IntEnum

class InputSymbol(IntEnum): # обнаружил/не обнаружил еду (входные данные)
    NOTFOUND = 0
    FOUND = 1

class OutputSymbol(IntEnum): # движение вперед, налево или направо (выходные данные)
    FORWARD = 0
    LEFT = 1
    RIGHT = 2

In [ ]:
!pip install plotly

In [ ]:
import plotly.graph_objects as go

def plot_map_and_path(map_data, route_data, moves):

    fig = go.Figure()

    passed_x = []
    passed_y = []
    for i in range(len(map_data)):
        for j in range(len(map_data[0])):
            if map_data[i][j] == "passed":
                passed_x.append(j)
                passed_y.append(len(map_data) - i - 1)

    if passed_x and passed_y:
        fig.add_trace(go.Scatter(x=passed_x, y=passed_y, mode='markers', marker=dict(color='blue'), name='Passed'))

        fig.add_trace(go.Scatter(x=[passed_x[-1]], y=[passed_y[-1]], mode='markers', marker=dict(color='green', size=10),
                                 name='Ant Position'))

    for i in range(len(map_data)):
        for j in range(len(map_data[0])):
            if map_data[i][j] == "food":
                fig.add_trace(go.Scatter(x=[j], y=[len(map_data) - i - 1], mode='markers', marker=dict(color='red', size=10), name='Food'))
            elif map_data[i][j] == "empty":
                fig.add_trace(go.Scatter(x=[j], y=[len(map_data) - i - 1], mode='markers', marker=dict(color='white', size=10), name='Empty'))

    fig.update_layout(xaxis_range=[-1, len(map_data[0])], yaxis_range=[-1, len(map_data)],
                      xaxis_scaleanchor='y', yaxis_scaleanchor='x', xaxis_fixedrange=True, yaxis_fixedrange=True,
                      xaxis_visible=False, yaxis_visible=False,
                      xaxis_ticks='', yaxis_ticks='',
                      showlegend=False,
                      width=1000, height=800,
                      plot_bgcolor='rgb(245, 245, 220)',
                      paper_bgcolor='rgb(245, 245, 220)')

    fig.show()

In [ ]:
def algorithm(population_size, generations, mutation_rate, elite_size, states_count, init_state, input_symbols, output_symbols):

    populations = [[] for _ in range(len(states_count))]

    for i, state_count in enumerate(states_count):
        population = [Machine.create_random(state_count, init_state, input_symbols, output_symbols) for _ in range(population_size)]
        populations[i].extend(population)

    for gen in range(generations):
        for i, population in enumerate(populations):
            fitness_scores = [(machine, Machine.evaluate(machine)) for machine in population]
            sorted_population = sorted(fitness_scores, key=lambda x: x[1], reverse=False)
            elite_machines = [machine for machine, _ in sorted_population[:elite_size]]

            next_population = elite_machines[:]
            while len(next_population) < population_size:
                parent1, parent2 = random.choices(population, k=2)
                child = Machine.crossover(parent1, parent2)
                Machine.mutate(child, mutation_rate)
                next_population.append(child)

            populations[i] = next_population

    best_machines = [min(population, key=lambda x: Machine.evaluate(x)) for population in populations]
    best_machine = min(best_machines, key=lambda x: Machine.evaluate(x))

    return best_machine

def main():

    antsimulator = AntSimulator(900)
    with open("map2.txt", 'r') as trial_file:
        antsimulator.parse_matrix(trial_file)

    print()
    print("Начальная карта:")
    plot_map_and_path(antsimulator.matrix_exc, antsimulator.position, antsimulator.moves)

    population_size = 100
    generations = 1000
    mutation_rate = 0.1
    elite_size = 2

    states_count = [4, 6, 8, 10, 12]
    input_symbols = [InputSymbol.NOTFOUND, InputSymbol.FOUND]
    output_symbols = [OutputSymbol.FORWARD, OutputSymbol.LEFT, OutputSymbol.RIGHT]
    init_state = 0

    best_machine = algorithm(population_size, generations, mutation_rate, elite_size, states_count, init_state, input_symbols, output_symbols)

    eaten_food = Machine.run_with_fsm(antsimulator, best_machine)
    fitness = Machine.evaluate(best_machine)
    state_count = len(best_machine.states)

    print("Лучший найденный автомат:")
    best_machine.print_transition_table()
    print("Количество состояний:", state_count)
    print("Количество съеденной еды:", eaten_food)
    print("Стоимость:", fitness)
    print("Путь муравья:")
    antsimulator.write_route("route.txt")

    plot_map_and_path(antsimulator.matrix_exc, antsimulator.position, antsimulator.moves)

if __name__ == "__main__":

    main()



Начальная карта:


Лучший найденный автомат:
	 InputSymbol.NOTFOUND|	InputSymbol.FOUND
---------------------------------------------------
0 | 	 3/OutputSymbol.RIGHT|	3/OutputSymbol.RIGHT
1 | 	 4/OutputSymbol.FORWARD|	3/OutputSymbol.FORWARD
2 | 	 0/OutputSymbol.FORWARD|	1/OutputSymbol.LEFT
3 | 	 2/OutputSymbol.LEFT|	2/OutputSymbol.FORWARD
4 | 	 1/OutputSymbol.RIGHT|	3/OutputSymbol.FORWARD
5 | 	 0/OutputSymbol.LEFT|	4/OutputSymbol.LEFT
---------------------------------------------------
Количество состояний: 6
Количество съеденной еды: 89
Стоимость: 955
Путь муравья:
